In [1]:
import os, sys, time, pickle
from pathlib import Path
from collections import Counter

PROJECT_DIR = Path(sys.path[0])/".."
DATA_DIR = PROJECT_DIR/"data"
SRC_DIR = PROJECT_DIR/"src"
DEPS_DIR = PROJECT_DIR/"deps"

sys.path.append(str(SRC_DIR))
sys.path.append(str(DEPS_DIR))

%pylab inline
%config InlineBackend.figure_format='retina'

%load_ext autoreload
%autoreload 2

from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from fairseq_wav2vec import Wav2Vec2Config, Wav2Vec2Model
import pandas as pd

from mugen_train import musicDataset, IDX_to_GENRE, GENRE_TO_IDX, get_music_data_loaders

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/keld/miniconda3/envs/mugen_ml/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


data

In [2]:
top50_genre_samples_df = pd.read_csv("/n1Tb/sc_mp3_top50_genre_samples.tsv_gz", compression='gzip', sep='\t')

train_loader, valid_loader = get_music_data_loaders(top50_genre_samples_df)



In [3]:
model = Wav2Vec2Model(Wav2Vec2Config)
model = model.to("cuda")

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, betas=(0.9,0.98), eps=1e-06, weight_decay=0.01, fused=True)

In [4]:
#model._parameters.__dict__

In [ ]:
data_loader = train_loader

model.train()
running_loss = 0.0
losses = []
for i, (batch, genres) in tqdm(enumerate(data_loader, 0), total=len(data_loader), disable=True):
    optimizer.zero_grad()
    outputs = model(batch)
    loss = outputs["features_pen"]
    loss.backward()
    optimizer.step()
    loss_item = loss.to("cpu").item() 
    running_loss += loss_item
    losses.append(loss_item)
    print(loss_item)
    
    param_names = list(model.state_dict().keys())
    for name, param in model.named_parameters():
        has_nan = torch.isnan(param).any()
        has_inf = torch.isinf(param).any()

        if has_nan or has_inf:
            print(f"Parameter {name} has_nan {has_nan} has_inf {has_inf}.")
            break
    if has_nan or has_inf:
        break



# return running_loss / len(data_loader)


In [ ]:
for name, param in model.named_parameters():
    print(name, param)



In [7]:
model.named_parameters()

<generator object Module.named_parameters at 0x7ff4728670b0>

In [13]:
running_loss

0.013600117526948452

In [ ]:
model

In [9]:
%debug

> /tmp/ipykernel_1539100/1953120967.py(12)<module>()
      9     loss.backward()
     10     optimizer.step()
     11     running_loss += loss.to("cpu").item()
---> 12     raise
     13 return running_loss / len(data_loader)



ipdb>  q



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/home/keld/miniconda3/envs/mugen_ml/lib/python3.9/bdb.py", line 359, in set_quit
    sys.settrace(None)



In [7]:
from ignite.handlers import FastaiLRFinder

trainer = ...
model = ...
optimizer = ...

lr_finder = FastaiLRFinder()
to_save = {"model": model, "optimizer": optimizer}

with lr_finder.attach(trainer, to_save=to_save) as trainer_with_lr_finder:
    trainer_with_lr_finder.run(dataloader)

# Get lr_finder results
lr_finder.get_results()

# Plot lr_finder results (requires matplotlib)
lr_finder.plot()

# get lr_finder suggestion for lr
lr_finder.lr_suggestion()


TypeError: Object <class 'ellipsis'> should have `state_dict` method

### Test 2 -- with small update

In [4]:
train_loses = []
val_loses = []
for epoch in range(3):
    train_loss = train(model, optimizer, train_loader)
    val_loss = validation(model, valid_loader)
    print(f"{epoch:03d} train_loss: {train_loss:0.4f} val_loss: {val_loss:0.4f}")
    train_loses.append(train_loss)
    val_loses.append(val_loss)

100%|████████████████████████████████████████████████████████████| 17/17 [00:04<00:00,  4.05it/s]


000 train_loss: 0.0000 val_loss: 0.0000


100%|████████████████████████████████████████████████████████████| 17/17 [00:04<00:00,  4.10it/s]


001 train_loss: 0.0000 val_loss: 0.0000


100%|████████████████████████████████████████████████████████████| 17/17 [00:04<00:00,  4.10it/s]

002 train_loss: 0.0000 val_loss: 0.0000


In [ ]:
plt.plot(train_loses, label="train_loses");
plt.plot(val_loses, label="val_loses");
plt.legend();

### Test 1 -- diverging

In [5]:
train_loses = []
val_loses = []
for epoch in range(5):
    train_loss = train(model, optimizer, train_loader)
    val_loss = validation(model, valid_loader)
    print(f"{epoch:03d} train_loss: {train_loss:0.4f} val_loss: {val_loss:0.4f}")
    train_loses.append(train_loss)
    val_loses.append(val_loss)

100%|████████████████████████████████████████████████████████████| 17/17 [00:04<00:00,  4.10it/s]


000 train_loss: 0.0000 val_loss: 0.0000


100%|████████████████████████████████████████████████████████████| 17/17 [00:04<00:00,  4.11it/s]


001 train_loss: 0.0000 val_loss: 0.0000


100%|████████████████████████████████████████████████████████████| 17/17 [00:04<00:00,  4.11it/s]


002 train_loss: 0.0000 val_loss: 0.0000


100%|████████████████████████████████████████████████████████████| 17/17 [00:04<00:00,  4.09it/s]


003 train_loss: 0.0000 val_loss: 0.0000


100%|████████████████████████████████████████████████████████████| 17/17 [00:04<00:00,  4.05it/s]

004 train_loss: 0.0000 val_loss: 0.0000


In [ ]:
%debug

#### Scratchpad

In [49]:
for epoch in range(10):
    train_loss = train(model, optimizer, criterion, train_loader)
    print(f"Epoch {epoch+1}, train loss: {train_loss:.3f}")


In [66]:
for i, batch in enumerate(top50_dataloader):
    break

In [18]:
out = model.forward(sample_bs)

In [160]:
sample_bs.shape

torch.Size([32, 120000])

In [144]:

d.shape

torch.Size([2, 273024])

In [4]:
model.eval()
running_loss = 0.0
for i, (batch, genres) in tqdm(enumerate(valid_loader, 0), total=len(valid_loader)):
    outputs = model(batch)
    break

  0%|                                                                     | 0/17 [00:02<?, ?it/s]


In [18]:
outputs['x'].shape

torch.Size([101, 12, 120])

In [10]:
batch

<function __main__.music_collate_fn_wrap.<locals>.music_collate_fn(x)>